In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/generative-dog-images/Annotation.zip
!unzip /kaggle/input/generative-dog-images/all-dogs.zip

In [ ]:
breeds = os.listdir('Annotation/')
breeds[0:5]

In [ ]:
import xml.etree.ElementTree as ET # for parsing XML
import matplotlib.pyplot as plt # to show images
from PIL import Image

In [ ]:
image_width = 64
image_height = 64
image_channels = 3
scale_factor = 16

dataset = []
dog_breed = []

for breed in breeds:
    for dog in os.listdir('Annotation/' + breed):
        try: img = Image.open('all-dogs/' + dog + '.jpg')
        except: continue
        tree = ET.parse('Annotation/' + breed + '/' + dog)
        root = tree.getroot()
        objects = root.findall('object')
        for o in objects:
            bndbox = o.find('bndbox') # reading bound box
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            w = np.min((xmax - xmin, ymax - ymin))
            img_cropped = img.crop((xmin, ymin, xmin+w, ymin+w))
            img_cropped = img_cropped.resize((image_width, image_height))
            dataset.append(np.asarray(img_cropped))
            dog_breed.append(breed.split('-')[1])

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(16,16))
for indx, axis in enumerate(axes.flatten()):
    i = np.random.randint(0, len(dataset))
    axis.set_axis_off()
    axis.imshow((dataset[i]).astype('uint8'))
    axis.text(0,-5,dog_breed[i])

In [ ]:
len(dataset)

In [ ]:
import tensorflow as tf
dog_features_tf = tf.cast(dataset, 'float32')

In [ ]:
def flip(x: tf.Tensor) -> (tf.Tensor):
    x = tf.image.random_flip_left_right(x)
    return x

In [ ]:
dog_features_data = tf.data.Dataset.from_tensor_slices(dog_features_tf).shuffle(22125).map(flip).batch(batch_size = 32, drop_remainder=True)

In [ ]:
dog_features_data

In [ ]:
for i in range(len(dataset)):
    dataset[i] = (dataset[i].astype(np.float32) - 127.5) / 127.5

In [ ]:
import keras
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, ReLU
from keras.backend import random_normal, ones_like, zeros_like, mean
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import concatenate
from keras.initializers import TruncatedNormal
from keras.callbacks import LearningRateScheduler, EarlyStopping, History

In [ ]:
# import tensorflow as tf
weight_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02, mean=0, seed=42)

In [ ]:
def transposed_conv(model, out_channels, ksize, stride_size, ptype='same'):
    model.add(Conv2DTranspose(out_channels, (ksize, ksize),
                              strides=(stride_size, stride_size), 
                              padding=ptype, 
                              kernel_initializer=weight_initializer, 
                              use_bias=False))
    model.add(BatchNormalization())
    model.add(ReLU())
    return model


def convSN(model, out_channels, ksize, stride_size):
    model.add(Conv2D(out_channels, (ksize, ksize), strides=(stride_size, stride_size), padding='same',
                     kernel_initializer=weight_initializer, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    return model

In [ ]:
def DogGenerator():
    model = Sequential()
    model.add(Dense(image_width // scale_factor * image_height // scale_factor * 128,
                    input_shape=(128,), kernel_initializer=weight_initializer))
    model.add(Reshape((image_height // scale_factor, image_width // scale_factor, 128)))
    
    model = transposed_conv(model, 512, ksize=5, stride_size=1)
    model.add(Dropout(0.3))
    model = transposed_conv(model, 256, ksize=5, stride_size=2)
    model.add(Dropout(0.3))
    model = transposed_conv(model, 128, ksize=5, stride_size=2)
    model = transposed_conv(model, 64, ksize=5, stride_size=2)
    model = transposed_conv(model, 32, ksize=5, stride_size=2)
    
    model.add(Dense(3, activation='tanh', kernel_initializer=weight_initializer))

    return model

dog_generator = DogGenerator()
dog_generator.summary()

In [ ]:
def DogDiscriminator():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(1,1), padding='same', use_bias=False,
                     input_shape=[image_height, image_width, image_channels], 
                     kernel_initializer=weight_initializer))
    model.add(LeakyReLU(alpha=0.2))

    model = convSN(model, 64, ksize=5, stride_size=2)
    model = convSN(model, 128, ksize=5, stride_size=2)
    model = convSN(model, 256, ksize=5, stride_size=2)
    model.add(Dropout(0.2))
    model = convSN(model, 512, ksize=5, stride_size=2)
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

dog_discriminator = DogDiscriminator()
dog_discriminator.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output): 
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        loss = fake_loss + real_loss
        return loss
    
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5)
discriminator_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.0002, beta1=0.5)

In [ ]:
epochs = 250
num_examples_to_generate = 1000

noise_dim = 128
batch_size = 32

In [ ]:
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = dog_generator(noise, training = True)
        
        real_output = dog_discriminator(images, training = True)
        fake_output = dog_discriminator(generated_images, training = True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
 
    gradients_of_generator = gen_tape.gradient(gen_loss, dog_generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, dog_discriminator.trainable_variables)
    

    generator_optimizer.apply_gradients(zip(gradients_of_generator, dog_generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, dog_discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
import time
from tqdm import tqdm

decay_step = 25

def decayed_learning_rate(lr, step, alpha = 0.1):
  step = min(step, decay_step)
  cosine_decay = 0.5 * (1 + cos(pi * step / decay_step))
  decayed = (1 - alpha) * cosine_decay + alpha
  return lr * decayed

def train(dataset, epochs):
    all_gl = np.array([]); all_dl = np.array([])
    for epoch in tqdm(range(epochs)):
        
        G_loss = []; D_loss = []
        
        start = time.time()
        new_lr_d = new_lr_g = 0.0002
        global_step = 0
        np.random.seed(seed=int(time.perf_counter()))
        
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            global_step = global_step + 1
            G_loss.append(g_loss); D_loss.append(d_loss)
            all_gl = np.append(all_gl,np.array([G_loss]))
            all_dl = np.append(all_dl,np.array([D_loss]))
        
        # Cosine learning rate decay
        if (epoch + 1) % decay_step == 0:
            new_lr_d = decayed_learning_rate(new_lr_d, global_step)
            new_lr_g = decayed_learning_rate(new_lr_g, global_step)
            generator_optimizer = tf.train.AdamOptimizer(learning_rate=new_lr_d, beta1=0.5)
            discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=new_lr_g, beta1=0.5)          

        print('Epoch: {} computed for {} sec'.format(epoch + 1, time.time() - start))
        print('Gen_loss mean: ', np.mean(G_loss),' std: ', np.std(G_loss))
        print('Disc_loss mean: ', np.mean(D_loss),' std: ', np.std(D_loss))
        

In [ ]:
dog_discriminator.compile(loss='binary_crossentropy',
    optimizer=generator_optimizer,
    metrics=['accuracy'])

dog_generator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)

train(batches, epochs)

In [ ]:
import os
z = zipfile.PyZipFile('images.zip', mode='w')
for k in range(num_examples_to_generate):
    generated_image = dog_generator(tf.random.normal([1, noise_dim]), training=False)
    f = str(k)+'.png'
    img = np.array(generated_image)
    img = (img[0, :, :, :] + 1.) / 2.
    img = Image.fromarray((255*img).astype('uint8').reshape((image_height,image_width,image_channels)))
    img.save(f,'PNG')
    z.write(f)
    os.remove(f)
z.close()
print('Saved final images for submission.')
